In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import string as st
import cv2

In [2]:
def tensorflow_shutup():
    """
    Make Tensorflow less verbose
    """
    try:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

        # noinspection PyPackageRequirements
        from tensorflow.python.util import deprecation

        tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

        # Monkey patching deprecation utils to shut it up! Maybe good idea to disable this once after upgrade
        # noinspection PyUnusedLocal
        def deprecated(date, instructions, warn_once=True):  # pylint: disable=unused-argument
            def deprecated_wrapper(func):
                return func
            return deprecated_wrapper

        deprecation.deprecated = deprecated

    except ImportError:
        pass

tensorflow_shutup()

In [11]:
# Create the data augmentation model
data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1, fill_mode='nearest'),
    layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode='nearest')
])

In [12]:
# Import the images from the asl_dataset and do augmentation for all classes
labels = st.digits
iters = 800
path = os.path.join('asl_alphabet_train', 'asl_alphabet_train')
for label in labels:
    imgs = os.listdir(os.path.join(path, label))
    data = []
    for img_name in imgs:
        img = Image.open(os.path.join(path, label, img_name)).resize((100, 100))
        arr = np.asarray(img)
        data.append(arr)
    data = np.array(data) # Original images of the dataset as arrays
    ind1 = 0
    while len(imgs) < iters:   
        aug_data = data_augmentation(data)
        aug_data = np.array(aug_data)
        for ind2, aug_d in enumerate(aug_data, 1):
            img = Image.fromarray(aug_d.astype('uint8'))
            img.save(os.path.join(path, label, '{}.jpeg'.format(str(ind1) + str(ind2))))
        imgs = os.listdir(os.path.join(path, label))
        ind1 += 1

In [22]:
# Image contouring
labels = st.digits + st.ascii_lowercase
for label in labels:
    imgs = os.listdir(os.path.join('asl_dataset', label))
    targ_dir = os.path.join('seg dataset', label)
    os.mkdir(targ_dir)
    for img_name in imgs:
        img = cv2.imread((os.path.join('asl_dataset', label, img_name)))
        # convert img to grey
        img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # set a thresh
        thresh = 40
        # get threshold image
        ret,thresh_img = cv2.threshold(img_grey, thresh, 255, cv2.THRESH_BINARY)
        # find contours
        contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # create an empty image for contours
        img_contours = np.zeros(img.shape)
        # draw the contours on the empty image
        cv2.drawContours(img_contours, contours, -1, (0,255,0), 3)
        # Fill the polygon with white
        cv2.fillPoly(img_contours, pts=contours, color=(255,255,255))
        # Save the image
        cv2.imwrite(os.path.join(targ_dir, img_name), img_contours)